# 5. Edge Computing with Raspberry Pi

## 5.1 Coral Edge TPU
* 구글에서 설계한 소형 ASIC(주문형 반도체)
* 저전력 정차의 고성능 머신러닝 추론 기능 제공
* Tensorflow Lite only
* 비전 기반 머신러닝 어플리케이션에 적합
* Backpropagration을 하지 못하기 때문에 딥러닝 훈련에 사용 불능
    * Transfer Learning 가능
* Edge TPU에 사용할 모델은 TF Lite에 맞게 번환해야 하고 양자화 필요
* 2가지 종류
    * Dev Board : SoC와 Edge TPU를 통합한 싱글보드 컴퓨터
    * USB Accelerator : USB 방식의 엑세서리 장치
    * https://coral.withgoogle.com/

## 5.2 설치
### 요구사항
* OS : Debian 6.0 이상 계열의 리눅스
* CPU : x86-64 또는 ARM32/64 ARMv8 명령셋
* Raspberry Pi 2/3 Model B/B+ 지원, Pi Zero는 비공식 지원
* Edge TPU Runtime and Python API 설치

### Edget TPU Runtime & Python Librar 설치
* `cd ~/`
* `wget https://dl.google.com/coral/edgetpu_api/edgetpu_api_latest.tar.gz -O edgetpu_api.tar.gz --trust-server-names `
* `tar xzf edgetpu_api.tar.gz `
* `cd edgetpu_api`
* `bash ./install.sh `
    * > maximum operation frequency?
        * Y : Inference 속도를 최대로 끌어 올린다. USB 장치가 매우 뜨거워지니 화상에 주의 할것!
        * N(Default) : 최대 속도 사용 안함
        * 변경하려면 install.sh을 다시 실행
    * 종속 라이브러리 설치, /lib/arm-linux-gnueabihf/libedgetpu.so 설치
* USB 장치 연결(만약 미리 연결했다면 다시 연결)
    * 전원 확인 : 장치 뒷면 LED(백색) 점등 확인
        * `lsusb` :
            * Bus 001 Device 004: ID 1a6e:089a Global Unichip Corp
    * 동작 중 : 장치 뒷면 LED 점멸
        * `lsusb` :
            * 

## 5.3 Demo 
### Demo 소스 코드
* `cd /usr/local/lib/python3.5/dist-packages/edgetpu/demo`
   * classify_image.py
   * classify_capture.py
   * object_detection.py
   * classification_transfer_learning.py
   * two_models_inference.py

### Pre-Comiled Model
* https://coral.withgoogle.com/models
* Image classification
    * MobileNet V1(ImageNet)
    * MobileNet V2(ImageNet)
    * MobileNet V2(iNat insercts)
    * MobileNet V2(iNat plants)
    * MobileNet V2(iNat birds)
    * Inception V1(ImageNet)
    * Inception V2(ImageNet)
    * Inception V3(ImageNet)
    * Inception V4(ImageNet)
* Object Detection
    * MobileNet SSD v1(COCO)
    * MobileNet SSD v2(COCO)
    * MobileNet SSD v2(Faces)
* Embedding Extractor
    * MobileNet v1
* 원하는 모델과 Label(text) 파일을 다운로드

### Image Classification
* Model & Label Download
    * `cd ~/Downloads`
    * `curl -O https://dl.google.com/coral/canned_models/mobilenet_v2_1.0_224_inat_bird_quant_edgetpu.tflite \
-O https://dl.google.com/coral/canned_models/inat_bird_labels.txt \
-O https://coral.withgoogle.com/static/images/parrot.jpg`
* 실행
    * `cd /usr/local/lib/python3.5/dist-packages/edgetpu/demo`
    * `python3 classify_image.py \
--model ~/Downloads/mobilenet_v2_1.0_224_inat_bird_quant_edgetpu.tflite \
--label ~/Downloads/inat_bird_labels.txt \
--image ~/Downloads/parrot.jpg`
* 결과
    * `---------------------------
Ara macao (Scarlet Macaw)
Score :  0.761719`

### Object Detection
* Model & Label Download
    * `cd ~/Downloads`
    * `curl -O https://dl.google.com/coral/canned_models/mobilenet_ssd_v2_face_quant_postprocess_edgetpu.tflite \
-O https://coral.withgoogle.com/static/images/face.jpg`
* feh 이미지 뷰어 설치
    * ` sudo apt-get install feh -y`
* 데모 실행
    * VNC 연결
    * `python3 object_detection.py \
--model ~/Downloads/mobilenet_ssd_v2_face_quant_postprocess_edgetpu.tflite \
--input ~/Downloads/face.jpg \
--output ~/detection_results.jpg`

* 실행 결과    
![detection_result](https://coral.withgoogle.com/static/images/detection_results.jpg)

### Realtime Camera classification
* Model download
    * `cd ~/Downloads`
    * `curl -O https://dl.google.com/coral/canned_models/mobilenet_v2_1.0_224_quant_edgetpu.tflite \
-O https://dl.google.com/coral/canned_models/imagenet_labels.txt`
* 데모 실행
    * `cd /usr/local/lib/python3.5/dist-packages/edgetpu/demo`
    * `python3 classify_capture.py \
--model ~/Downloads/mobilenet_v2_1.0_224_quant_edgetpu.tflite \
--label ~/Downloads/imagenet_labels.txt`

## Edge TPU Python API
* edetpu.basic.basic_engine : https://coral.withgoogle.com/docs/reference/edgetpu.basic.basic_engine/
    * BasicEngine
    * Tensorflow Lite Model을 Edge TPU에서 실행하기 위한 기반 엔진
* edgetpu.basic.edgetpu_utils : https://coral.withgoogle.com/docs/reference/edgetpu.basic.edgetpu_utils/
    * Utility 함수와 상수
* edgetpu.classification.engine : https://coral.withgoogle.com/docs/reference/edgetpu.classification.engine/
    * ClassificationEngine
    * 이미지 분류를 위한 BasicEngie 확장 클래스
* edgetpu.detection.engine : https://coral.withgoogle.com/docs/reference/edgetpu.detection.engine/
    * DetectionCandidate
        * detection 후보 정보를 위한 구조체
    * DetectionEngine
        * Object Detection을 위한 BasicEngine 확장 클래스
* edgetpu.learn.imprinting.engine : https://coral.withgoogle.com/docs/reference/edgetpu.learn.imprinting.engine/
    * ImprintingEngine
    * 이미지 분류 transfer-learning을 위한 엔진
* edgetpu.utils.image_process : https://coral.withgoogle.com/docs/reference/edgetpu.utils.image_processing/
    * ResamplingWithOriginalRatio
    * 이미지 전처리를 위한 유틸릴티
    


## Realtime Object Detection 예제
### MobileNet SSD V2(Coco) Sync
* 모델 다운로드
    * `curl -O https://dl.google.com/coral/canned_models/mobilenet_ssd_v2_coco_quant_postprocess_edgetpu.tflite`
    * `curl -O https://dl.google.com/coral/canned_models/coco_labels.txt`
* 실행  
    * `python3 cam_detect_sync.py`
* 소스 코드

```python
import argparse
import platform
import numpy as np
import cv2
import time
from PIL import Image
from edgetpu.detection.engine import DetectionEngine


# Function to read labels from text files.
def ReadLabelFile(file_path):
  with open(file_path, 'r') as f:
    lines = f.readlines()
  ret = {}
  for line in lines:
    pair = line.strip().split(maxsplit=1)
    ret[int(pair[0])] = pair[1].strip()
  return ret


def main():
    parser = argparse.ArgumentParser()
    parser.add_argument("--model", default="mobilenet_ssd_v2_coco_quant_postprocess_edgetpu.tflite", help="Path of the detection model.")
    parser.add_argument("--label", default="coco_labels.txt", help="Path of the labels file.")
    parser.add_argument("--usbcamno", type=int, default=0, help="USB Camera number.")
    args = parser.parse_args()

    fps = ""
    detectfps = ""
    framecount = 0
    detectframecount = 0
    time1 = 0
    time2 = 0
    box_color = (255, 128, 0)
    box_thickness = 1
    label_background_color = (125, 175, 75)
    label_text_color = (255, 255, 255)
    percentage = 0.0

    camera_width = 320
    camera_height = 240

    cap = cv2.VideoCapture(args.usbcamno)
    cap.set(cv2.CAP_PROP_FPS, 150)
    cap.set(cv2.CAP_PROP_FRAME_WIDTH, camera_width)
    cap.set(cv2.CAP_PROP_FRAME_HEIGHT, camera_height)

    # Initialize engine.
    engine = DetectionEngine(args.model)
    labels = ReadLabelFile(args.label) if args.label else None

    while True:
        t1 = time.perf_counter()

        ret, color_image = cap.read()
        if not ret:
            break

        # Run inference.
        prepimg = color_image[:, :, ::-1].copy()
        prepimg = Image.fromarray(prepimg)

        tinf = time.perf_counter()
        ans = engine.DetectWithImage(prepimg, threshold=0.5, keep_aspect_ratio=True, relative_coord=False, top_k=10)
        print(time.perf_counter() - tinf, "sec")


        # Display result.
        if ans:
            detectframecount += 1
            for obj in ans:
                box = obj.bounding_box.flatten().tolist()
                box_left = int(box[0])
                box_top = int(box[1])
                box_right = int(box[2])
                box_bottom = int(box[3])
                cv2.rectangle(color_image, (box_left, box_top), (box_right, box_bottom), box_color, box_thickness)

                percentage = int(obj.score * 100)
                label_text = labels[obj.label_id] + " (" + str(percentage) + "%)" 

                label_size = cv2.getTextSize(label_text, cv2.FONT_HERSHEY_SIMPLEX, 0.5, 1)[0]
                label_left = box_left
                label_top = box_top - label_size[1]
                if (label_top < 1):
                    label_top = 1
                label_right = label_left + label_size[0]
                label_bottom = label_top + label_size[1]
                cv2.rectangle(color_image, (label_left - 1, label_top - 1), (label_right + 1, label_bottom + 1), label_background_color, -1)
                cv2.putText(color_image, label_text, (label_left, label_bottom), cv2.FONT_HERSHEY_SIMPLEX, 0.5, label_text_color, 1)

        cv2.putText(color_image, fps,       (camera_width-170,15), cv2.FONT_HERSHEY_SIMPLEX, 0.5, (38,0,255), 1, cv2.LINE_AA)
        cv2.putText(color_image, detectfps, (camera_width-170,30), cv2.FONT_HERSHEY_SIMPLEX, 0.5, (38,0,255), 1, cv2.LINE_AA)

        cv2.namedWindow('USB Camera', cv2.WINDOW_AUTOSIZE)
        cv2.imshow('USB Camera', color_image)

        if cv2.waitKey(1)&0xFF == ord('q'):
            break

        # FPS calculation
        framecount += 1
        if framecount >= 15:
            fps       = "(Playback) {:.1f} FPS".format(time1/15)
            detectfps = "(Detection) {:.1f} FPS".format(detectframecount/time2)
            framecount = 0
            detectframecount = 0
            time1 = 0
            time2 = 0
        t2 = time.perf_counter()
        elapsedTime = t2-t1
        time1 += 1/elapsedTime
        time2 += elapsedTime

if __name__ == '__main__':
    main()
```



### MobileNet SSD V2(Coco) ASync
* 쓰레드로 속도 개선
* 모델 다운로드
    * `curl -O https://dl.google.com/coral/canned_models/mobilenet_ssd_v2_coco_quant_postprocess_edgetpu.tflite`
    * `curl -O https://dl.google.com/coral/canned_models/coco_labels.txt`
* 실행  
    * `python3 cam_detect_async.py`
* 소스 코드

```python
import argparse
import platform
import numpy as np
import cv2
import time
from PIL import Image
from time import sleep
import multiprocessing as mp
from edgetpu.detection.engine import DetectionEngine

lastresults = None
processes = []
frameBuffer = None
results = None
fps = ""
detectfps = ""
framecount = 0
detectframecount = 0
time1 = 0
time2 = 0
box_color = (255, 128, 0)
box_thickness = 1
label_background_color = (125, 175, 75)
label_text_color = (255, 255, 255)
percentage = 0.0

# Function to read labels from text files.
def ReadLabelFile(file_path):
  with open(file_path, 'r') as f:
    lines = f.readlines()
  ret = {}
  for line in lines:
    pair = line.strip().split(maxsplit=1)
    ret[int(pair[0])] = pair[1].strip()
  return ret


def camThread(label, results, frameBuffer, camera_width, camera_height, vidfps, usbcamno):

    global fps
    global detectfps
    global framecount
    global detectframecount
    global time1
    global time2
    global lastresults
    global cam
    global window_name

    cam = cv2.VideoCapture(usbcamno)
    cam.set(cv2.CAP_PROP_FPS, vidfps)
    cam.set(cv2.CAP_PROP_FRAME_WIDTH, camera_width)
    cam.set(cv2.CAP_PROP_FRAME_HEIGHT, camera_height)
    window_name = "USB Camera"
    cv2.namedWindow(window_name, cv2.WINDOW_AUTOSIZE)

    while True:
        t1 = time.perf_counter()

        ret, color_image = cam.read()
        if not ret:
            continue
        if frameBuffer.full():
            frameBuffer.get()
        frames = color_image
        frameBuffer.put(color_image.copy())
        res = None

        if not results.empty():
            res = results.get(False)
            detectframecount += 1
            imdraw = overlay_on_image(frames, res, label, camera_width, camera_height)
            lastresults = res
        else:
            imdraw = overlay_on_image(frames, lastresults, label, camera_width, camera_height)

        cv2.imshow('USB Camera', imdraw)

        if cv2.waitKey(1)&0xFF == ord('q'):
            break

        # FPS calculation
        framecount += 1
        if framecount >= 15:
            fps       = "(Playback) {:.1f} FPS".format(time1/15)
            detectfps = "(Detection) {:.1f} FPS".format(detectframecount/time2)
            framecount = 0
            detectframecount = 0
            time1 = 0
            time2 = 0
        t2 = time.perf_counter()
        elapsedTime = t2-t1
        time1 += 1/elapsedTime
        time2 += elapsedTime



def inferencer(results, frameBuffer, model, camera_width, camera_height):

    engine = DetectionEngine(model)

    while True:

        if frameBuffer.empty():
            continue

        # Run inference.
        color_image = frameBuffer.get()
        prepimg = color_image[:, :, ::-1].copy()
        prepimg = Image.fromarray(prepimg)

        tinf = time.perf_counter()
        ans = engine.DetectWithImage(prepimg, threshold=0.5, keep_aspect_ratio=True, relative_coord=False, top_k=10)
        print(time.perf_counter() - tinf, "sec")
        results.put(ans)



def overlay_on_image(frames, object_infos, label, camera_width, camera_height):

    color_image = frames

    if isinstance(object_infos, type(None)):
        return color_image
    img_cp = color_image.copy()

    for obj in object_infos:
        box = obj.bounding_box.flatten().tolist()
        box_left = int(box[0])
        box_top = int(box[1])
        box_right = int(box[2])
        box_bottom = int(box[3])
        cv2.rectangle(img_cp, (box_left, box_top), (box_right, box_bottom), box_color, box_thickness)

        percentage = int(obj.score * 100)
        label_text = label[obj.label_id] + " (" + str(percentage) + "%)" 

        label_size = cv2.getTextSize(label_text, cv2.FONT_HERSHEY_SIMPLEX, 0.5, 1)[0]
        label_left = box_left
        label_top = box_top - label_size[1]
        if (label_top < 1):
            label_top = 1
        label_right = label_left + label_size[0]
        label_bottom = label_top + label_size[1]
        cv2.rectangle(img_cp, (label_left - 1, label_top - 1), (label_right + 1, label_bottom + 1), label_background_color, -1)
        cv2.putText(img_cp, label_text, (label_left, label_bottom), cv2.FONT_HERSHEY_SIMPLEX, 0.5, label_text_color, 1)

    cv2.putText(img_cp, fps,       (camera_width-170,15), cv2.FONT_HERSHEY_SIMPLEX, 0.5, (38,0,255), 1, cv2.LINE_AA)
    cv2.putText(img_cp, detectfps, (camera_width-170,30), cv2.FONT_HERSHEY_SIMPLEX, 0.5, (38,0,255), 1, cv2.LINE_AA)

    return img_cp

if __name__ == '__main__':

    parser = argparse.ArgumentParser()
    parser.add_argument("--model", default="mobilenet_ssd_v2_coco_quant_postprocess_edgetpu.tflite", help="Path of the detection model.")
    parser.add_argument("--label", default="coco_labels.txt", help="Path of the labels file.")
    parser.add_argument("--usbcamno", type=int, default=0, help="USB Camera number.")
    args = parser.parse_args()

    model    = args.model
    label    = ReadLabelFile(args.label)
    usbcamno = args.usbcamno

    camera_width = 320
    camera_height = 240
    vidfps = 30

    try:
        mp.set_start_method('forkserver')
        frameBuffer = mp.Queue(10)
        results = mp.Queue()

        # Start streaming
        p = mp.Process(target=camThread,
                       args=(label, results, frameBuffer, camera_width, camera_height, vidfps, usbcamno),
                       daemon=True)
        p.start()
        processes.append(p)

        # Activation of inferencer
        p = mp.Process(target=inferencer,
                       args=(results, frameBuffer, model, camera_width, camera_height),
                       daemon=True)
        p.start()
        processes.append(p)

        while True:
            sleep(1)

    finally:
        for p in range(len(processes)):
            processes[p].terminate()
```

## Tensorflow on Edge TPU

* https://coral.withgoogle.com/docs/edgetpu/models-intro/
* Edge TPU에 호환되는 모델 만드는 방법에 대해 설명한다.
* Quantization(양자화) : Edge TPU에 필수요건
    * 32bit float를 8bit fixed-point로 변경
    * 모델을 작고 빠르게 한다. 모델 표현의 정확도는 떨어 질 수 있지만, 신경망의 추론에는 영향이 별로 없다. 
*  전체 흐름(아래 그림)
![edge tpu](https://coral.withgoogle.com/static/images/compile-workflow.png)

### 모델 컴파일
* TF-Lite로 convert 한 모델을 컴파일 해야 Edge TPU에서 돌아 간다.
* https://coral.withgoogle.com/web-compiler/
    * TF-Lite로 convert 한 모델을 업로드해서 컴파일 진행
* 컴파일 요구 사항
    * 텐서 파라미터 양자화(8비트 고정 소수점 숫자), 아래 링크 이용
        * https://github.com/tensorflow/tensorflow/tree/r1.13/tensorflow/contrib/quantize#quantization-aware-training
        * post-training quantization 지원 안됨( https://www.tensorflow.org/lite/performance/post_training_quantization)
            * 8bit 고정길이로 줄인 모델도 inference 하는 동안 32bit 부동소수점이 되기 때문이다.
    * 텐서 사이즈는 컴파일 타임에 고정(다이나믹 사이즈 불가)
    * 바이어스 같은 모델 파라미터는 상수화, 컴파일 타임에
    * 텐서는 1,2,3차원 중에 하나만 가능
    * 모델은 Edge TPU에서 지원하는 연산만 사용
        * https://coral.withgoogle.com/docs/edgetpu/models-intro/#ops-table
* 만약 요구 사항을 만족하지 못하면 컴파일은 되지만 실행될때 일부분만 Edge TPU에서 실행되고 그렇지 않은 부분은 CPU에서 실행된다.
    * 모델은 1회만 구분할 수 있어서 미지원 연산을 사용한 이후 모든 부분은 CPU에서 실행된다.
    * visualize.py 같은 도구로 조사해 볼 수 있다.
        * https://github.com/tensorflow/tensorflow/blob/master/tensorflow/lite/tools/visualize.py

### Transfer learning
* 전체 흐름을 다 따를 필요없이 이미 존재하는 Edge TPU 호환 모델을  재훈련해서 사용 가능
* fine tuning 이라고도 부름
* 모델 전체의 weights와 bias를 재 훈련 할 수 도 있지만,
* classification을 실행하는 마지막 레이어만 제거하고 새로운 클래스를 인식하는 새로운 레이러만 훈련해서도 할 수 있다.
* 이미 훈련된 MobileNet에 새로운 클래스를 추가하는 튜토리얼이 있다.
* Edge TPU에서 재훈련도 가능하다.
   
### Retrain an Image Classification on Device
* 장점
    * 거의 실시간으로 transfer-learning이 엣지 장치에서 일어난다.
    * 모델을 재컴파일 할 필요 없다.
* 단점
    * 클래스당 최대 200개 이미지로 훈련 데이타 사이즈가 제한된다.
    * 작은 내부적 클래스 변화만 갖는 데이타셋 에만 의미가 있다.
    * 최종 fully connected layer는 CPU에서 돌아 가기 때문에 pre-compile 된 모델을 돌리는 것 보다 효과가 덜하다.
* 주요 방법
    * ImprintingEngine API 사용
    * classification_transfer_learning.py 예제 스크립트 제공
* 데모 작업 순서
    * 꽃 데이타셋 다운로드 및 압축해제
        * `wget http://download.tensorflow.org/example_images/flower_photos.tgz`
        * `tar -xvfz flower_photos.tgz`
    * embedding extractor 다운로드
        * `wget http://storage.googleapis.com/cloud-iot-edge-pretrained-models/canned_models/mobilenet_v1_1.0_224_quant_embedding_extractor_edgetpu.tflite`
    * transfer learning 시작
        * `python3 /usr/local/lib/python3.5/dist-packages/edgetpu/demo/classification_transfer_learning.py \
        --extractor mobilenet_v1_1.0_224_quant_embedding_extractor_edgetpu.tflite \
        --data flower_photos \
        --output flower_model.tflite --test_ratio 0.95`
        * 1~2분 소요 후에 flower_model.tflite 파일이 생성된다.
* 데모 실행
    * `wget -O rose.jpg https://c2.staticflickr.com/4/3062/3067374593_f2963e50b7_o.jpg`
    * `python3 /usr/local/lib/python3.5/dist-packages/edgetpu/demo/classify_image.py \
     --model flower_model.tflite \
     --label flower_model.txt \
     --image rose.jpg`
* 주의
    * 반드시 재훈련할 모델의 embedding extractor version을 써야 한다.
    * classification_transfer_learning.py 는 각 클래스별로 디렉토리를 만들어야 한다.
    * API는 클래스별로 최대 200개의 훈련이미지만 지원한다.
    * test_tatio flag로 훈련과 테스트 이미지 비율을 제어할 수 있다.

